## Gold layer Build Initial tables for model

In [1]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession

StatementMeta(, 3fc848f7-8f15-4d34-b18d-ad1bfa202080, 3, Finished, Available, Finished)

In [2]:
data_df = spark.read.table("events_silver")##.filter(col('time') > start_date)

StatementMeta(, 3fc848f7-8f15-4d34-b18d-ad1bfa202080, 4, Finished, Available, Finished)

In [4]:
display(data_df)

StatementMeta(, a9c1d626-1715-4ead-ac37-0aa020d34e88, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 31f843d4-e16d-4c76-8871-558f4fa3dc9a)

In [3]:
from pyspark.sql.functions import monotonically_increasing_id, year, month, dayofmonth, hour, minute, second

# Create and populate Place Dimension
place_df = data_df.select("place", "country_code").distinct().withColumn("place_id", monotonically_increasing_id())
place_df.write.mode("overwrite").saveAsTable("Place_Dimension")




StatementMeta(, 3fc848f7-8f15-4d34-b18d-ad1bfa202080, 5, Finished, Available, Finished)

In [4]:
# Create and populate Magnitude Category Dimension
mag_cat_df = data_df.select("magCategory", "sig_class", "magType").distinct().withColumn("magCategory_id", monotonically_increasing_id())
mag_cat_df.write.mode("overwrite").saveAsTable("Magnitude_Category_Dimension")


StatementMeta(, 3fc848f7-8f15-4d34-b18d-ad1bfa202080, 6, Finished, Available, Finished)

In [5]:
# Create and populate Time Dimension
from pyspark.sql.functions import col, year, month, dayofmonth, date_format, quarter

# Create and populate Time Dimension
time_df = data_df.select(
    col("time").alias("full_date").cast("date")
).distinct() \
    .withColumn("year", year(col("full_date"))) \
    .withColumn("month", month(col("full_date"))) \
    .withColumn("day", dayofmonth(col("full_date"))) \
    .withColumn("month_name", date_format(col("full_date"), "MMMM")) \
    .withColumn("week_name", date_format(col("full_date"), "EEEE")) \
    .withColumn("quarter", quarter(col("full_date")))
time_df.write.mode("overwrite").saveAsTable("Time_Dimension")

StatementMeta(, 3fc848f7-8f15-4d34-b18d-ad1bfa202080, 7, Finished, Available, Finished)

In [32]:
display(time_df)

StatementMeta(, a9c1d626-1715-4ead-ac37-0aa020d34e88, 34, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a1425a7d-bac2-4862-a9dd-b167f6b176bc)

In [6]:

# Create and populate Fact Table
fact_df = data_df.withColumn("full_date", col("time").cast("date")) \
    .join(place_df, ["place", "country_code"], "left") \
    .join(mag_cat_df, ["magCategory", "sig_class", "magType"], "left") \
    .join(time_df, "full_date", "left") \
    .select(
        col("magnitude"),
        col("latitude"),
        col("longitude"),
        col("depth"),
        col("sig"),
        col("updated"),
        col("place_id"),
        col("magCategory_id"),
        col("full_date")
    )
fact_df.write.mode("overwrite").saveAsTable("Earthquake_Fact")

StatementMeta(, 3fc848f7-8f15-4d34-b18d-ad1bfa202080, 8, Finished, Available, Finished)

In [7]:
display(fact_df)

StatementMeta(, 3fc848f7-8f15-4d34-b18d-ad1bfa202080, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c6c15a90-e25a-46ab-b1ac-dbab7e948b13)

In [8]:
print("All",data_df.count()) #382889
print("Time", time_df.count() )
print("mag_cat_df", mag_cat_df.count() )
print("place_df", place_df.count() )
print("fact_df", fact_df.count() )


StatementMeta(, 3fc848f7-8f15-4d34-b18d-ad1bfa202080, 10, Finished, Available, Finished)

All 1504
Time 6
mag_cat_df 18
place_df 1011
fact_df 1504
